# Modelling

Now that we have studied the data and acquired a basic understanding of wind turbine function, we are ready to build some models.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import os

from sqlalchemy import create_engine
from dotenv import load_dotenv

## Approach

My suggestion for Engie is to build a 'digital twin' of the wind turbine. This is a digital representation of the wind turbine which we can use to simulate the behaviour of the wind turbine under a variety of conditions, allowing us to predict future behaviour, given a weather forecast, as well as identify when certain attributes of the wind turbine are not behaving as expected.

### Theory

A digital twin can be conceived as a big 'lookup table' that maps observed inputs into expected outputs. These inputs can be individual parameters, or a set of parameters. For example, we may wish to 'look up' the expected power output of the wind turbine, given a certain wind speed (this would give the so-called *power curve*, discussed later), or instead look up the expected rotor torque given a certain wind speed, rotor speed and air pressure.

We can think of this lookup table as a big joint probability distribution over the entire state space of the wind turbine. Querying the 'lookup table' is then equivalent to computing conditional probabilities over the state space. This object is highly valuable! We would expect to see probability mass concentrated around key regions in the state space; for example, high wind speeds in the afternoon should be associated with high power output, and low wind speeds at night should be associated with low power output.

So! To model this joint probability distribution, we could just fit a big Gaussian mixture model to the data, in some suitably transformed space. However, this state space is very high-dimensional, and probably computationally intractable. Moreover, we know that there are important causual relationships between the different parameters, which are determined by the known physical laws that govern the function of the wind turbine. Hence a more efficient and principled approach would be to use a Bayesian network, which allows us to encode these causal relationships in a graphical model.

The lookup table should also in some sense be dynamic, and able to be updated as new data is collected... that is, we should be able to incrementally evolve the JPD as new data is collected. This is a key feature of a digital twin! But we will have to do so carefully, in some sense 'skeptically'... and this can be done with a Dynamic Bayesian Network. But I won't be looking at these, for now.

So: the goal is to model each wind turbine as a Bayesian network, using the historical data to learn the parameters of the network. We can then use this network to make predictions about future behaviour, given a weather forecast, and also to identify when the wind turbine is not behaving as expected.

### Applications

There two immediate applications of this lookup table: anomaly detection and forecasting.

#### Anomaly detection

If we observe a state that has low probability mass, we can flag this as an anomaly. For example, high rotor torque at low wind speeds may indicate a mechanical fault. Conversely, if we know that a certain attribute should be behaving in a certain way, we can flag regions of conditionally lower probability mass. One attribute that may be of particular interest is the *power curve*, which is specified and guaranteed by the manufacturer. Per (Hau, 2005),

> The power curve is a wind turbine’s official certificate of performance

This sounds a lot like ship performance curves! By modelling and monitoring these power curves, we could help Engie ensure that their wind turbines are performing as expected, and identify any deviations rated performance that may be due to mechanical faults or other issues.

#### Forecasting

If we have a weather forecast, we can use the lookup table to predict the expected behaviour of the wind turbine, both with a point estimate and a confidence interval.

### Implementation

To make this discussion a little more tangible, we can specify a rough outline of the desired API for interacting with this digital twin. Ideally, it would look something like:

```python
import DigitalTwin from toqua

# Create a new wind turbine
wt = DigitalTwin(type="wind_turbine")

# Train the wind turbine on some data
wt.train(data)

# Query the wind turbine
wt.query(
    inputs={
        'wind_speed': 10,
        'rotor_speed': 10,
        'air_pressure': 1000
    },
    output='power'
)
# -> returns a probability distribution over power (and a MAP estimate)
wt.query(
    inputs={
        'wind_speed': 10,
    },
    output='power'
)
# -> returns a probability distribution over power (and a MAP estimate)
# Assess an observation for anomalous behaviour
wt.assess(
    inputs={
        'wind_speed': 10,
        'rotor_speed': 10,
        'air_pressure': 1000,
        'power': 1000
    }
)
# -> returns a judgement on whether the observation is anomalous

# Update the twin with new data
wt.update(data)
```

## Modelling

So let's start experimenting with some Bayesian networks.

In [2]:
load_dotenv()
DATABASE_URL = os.getenv("DATABASE_URL")

engine = create_engine(DATABASE_URL)
conn = engine.connect()

In [3]:
df: pd.DataFrame = pd.read_sql("SELECT * FROM wind_turbines.observations_clean", conn)
df['datetime'] = pd.to_datetime(df['datetime'], utc=True)

Except oh no! I can't find any existing Python libraries that satisfactorily implement continuous Bayesian networks. There are a few that implement discrete Bayesian networks... but this seems like a very suboptimal approach. So for the sake of time, we are left to pursue some more conventional modelling approaches.

## Predictive Models

What can we do instead... let's just see if we can at least build some robust predictive models. If this model is strongly predictive, we can still use it to identify anomalous behaviour (though in a far less elegant fashion).

### A Simple Model

To start with, let us construct a model of the wind turbine's active power output, as a function of the weather. We know that there is a direct causual relationship between these two observables, so we should be able to attain strong model performance.

We can denote this simple model as

$$ P = f(W) $$

where $P \in \mathcal{P} \subset \mathbb{R}$ is the active power output, $W \in \mathcal{W} \subset \mathbb{R}^5$ is a vector of weather variables, composed of wind speed, wind direction (away from the nacelle), temperature, air pressure and humidity, and $f: \mathcal{W} \rightarrow \mathbb{R}$ is a representation of the wind turbine system.

However, we will immediately run into a problem here! The wind turbine system does not run in a vacuum – it is also attached to the power grid, meaning that the observed real power generated will also be a funtion of grid demand. But we do not have access to grid demand data...

We can, however, use the reactive power output as a proxy (i.e. instrumental variable) for the grid demand, since:
* Reactive power is correlated with grid demand (I think...)
* Reactive power is not nominally correlated with the weather, and a priori, not correlated with active power.
Our updated model is therefore

$$ P = f(W, Q) $$

where $Q \in \mathcal{Q} \subset \mathbb{R}$ is the reactive power output.

So let's construct some simple models of this form, and see how they perform.

In [4]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error

def print_scores(model, metric, X_train, y_train, X_test, y_test):
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    print(f"Train {metric.__name__}: {metric(y_train, y_pred_train):.4}")
    print(f"Test {metric.__name__}: {metric(y_test, y_pred_test):.4}")

In [5]:
weather_vars: list[str] = [
    'ws', 'va', 'temp', 'pressure', 'humidity',
]
covariates: list[str] = weather_vars + ['q']

X_train, X_test, y_train, y_test = train_test_split(
    df[covariates], df['p'], test_size=0.2, random_state=42
)

#### 1. Linear Model

Always a sensible place to start.

In [6]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

print_scores(model, r2_score, X_train, y_train, X_test, y_test)
print_scores(model, mean_squared_error, X_train, y_train, X_test, y_test)


Train r2_score: 0.8045
Test r2_score: 0.8064
Train mean_squared_error: 3.64e+04
Test mean_squared_error: 3.645e+04


Not bad, given how simple the model is.

#### 2. XGBoost

Consistently SOA.

In [7]:
import xgboost as xgb

model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
)
model.fit(X_train, y_train)

print_scores(model, r2_score, X_train, y_train, X_test, y_test)
print_scores(model, mean_squared_error, X_train, y_train, X_test, y_test)


Train r2_score: 0.9901
Test r2_score: 0.9898
Train mean_squared_error: 1.84e+03
Test mean_squared_error: 1.914e+03


So we have an out-of-the box $R^2$ of 0.99, which is very good! But we can likely still do better, if we apply some more domain knowledge and use some of the additional features we have engineered.

### A Physics-Informed Model

Let's now build a more sophisticated mode, informed by the physics of wind turbines and the features that we have previously curated. We know that active power generation is closely related to rotor torque (almost perfectly)... 

In [8]:
torque_to_power_model = LinearRegression()
torque_to_power_model.fit(df[['rm']], df['p'])
torque_to_power_model.score(df[['rm']], df['p'])

0.991439344143642

...and that rotor torque is explicitly a function of wind speed and air density. In particular, we know from (Hau 2005), we know that rotor torque increases in the square of the wind velocity, and linearly in air density. So let's include these variables directly in our model.

In [9]:
covariates: list[str] = [
    'ws', 'ws_sq', 'ws_cb', 'ws_a',
    'va',
    'temp', 'temp_6h',
    'pressure', 'humidity', 'rho',
    'o', 'rbt'
]

X_train, X_test, y_train, y_test = train_test_split(
    df[covariates], df['p'], test_size=0.2, random_state=42
)

#### 1. Linear Model

In [10]:
model = LinearRegression()
model.fit(X_train, y_train)

print_scores(model, r2_score, X_train, y_train, X_test, y_test)
print_scores(model, mean_squared_error, X_train, y_train, X_test, y_test)

Train r2_score: 0.9248
Test r2_score: 0.9268
Train mean_squared_error: 1.4e+04
Test mean_squared_error: 1.378e+04


Already we see a dramatic improvement, even in just the linear model.

#### 2. XGBoost

Let's do a proper grid search this time.

In [11]:
xgb_reg = xgb.XGBRegressor(
    objective='reg:squarederror',
    subsample=0.8,
    colsample_bytree=0.8,
)

params = {
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200, 500],
}

grid = GridSearchCV(xgb_reg, params, cv=3, n_jobs=-1, verbose=2)
grid.fit(X_train, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] END ......................max_depth=3, n_estimators=100; total time=  29.1s
[CV] END ......................max_depth=5, n_estimators=100; total time=  48.1s
[CV] END ......................max_depth=5, n_estimators=500; total time= 4.0min
[CV] END ......................max_depth=3, n_estimators=200; total time=  56.5s
[CV] END ......................max_depth=5, n_estimators=100; total time=  48.3s
[CV] END ......................max_depth=5, n_estimators=500; total time= 3.9min
[CV] END ......................max_depth=3, n_estimators=500; total time= 2.3min
[CV] END ......................max_depth=5, n_estimators=500; total time= 3.7min
[CV] END ......................max_depth=3, n_estimators=500; total time= 2.3min
[CV] END ......................max_depth=7, n_estimators=100; total time= 1.2min
[CV] END ......................max_depth=7, n_estimators=200; total time= 2.2min


In [ ]:
model = grid.best_estimator_

print_scores(model, r2_score, X_train, y_train, X_test, y_test)
print_scores(model, mean_squared_error, X_train, y_train, X_test, y_test)

Train r2_score: 0.9918
Test r2_score: 0.9844
Train mean_squared_error: 1.52e+03
Test mean_squared_error: 2.916e+03


Looks like we might be overfitting a little bit, but we are still obtaining high performance.

## Conclusions

So what have we learned?

* Continuous Bayesian networks are hard are great but there are no good Python libraries for them!
* We can build a simple model of wind turbine power output, using only weather data, that is highly predictive. This is a very tractable problem!
* This model can be used both for forecasting and anomaly detection (though I haven't demonstrated this here).


## Future Improvements

* Build a library for continuous Bayesian networks!
* Get a better measure of grid demand, such that we can isolate the effect of the weather on active power output.


## References

* https://www.sciencedirect.com/science/article/abs/pii/S0951832020305548
* https://www.youtube.com/watch?v=ZuSx0pYAZ_I